In [1]:
#This just does nice interpolations from the ephemeris data for the Sun and moon
using Interpolations
function ephemInterp(t,x1,x2,x3,x4,x5,x6)
x = [x1[t], x2[t], x3[t], x4[t], x5[t], x6[t]];
end

INFO: Recompiling stale cache file /Users/zacman/.julia/lib/v0.6/Interpolations.ji for module Interpolations.


ephemInterp (generic function with 1 method)

In [2]:
#This is the dynamics function for the spacecraft.
#It accounts for 1/r^2 gravity from the Earth, Sun, and moon.
function dynamics(xdot,x,ephem,t)

#Gravitational parameters (GM) in 1000*km^3/day^2
mu_s = 1.3271244004193938e11*((60*60*24)^2/(1000^3));
mu_e = 398600.440*((60*60*24)^2/(1000^3));
mu_m = 4902.80007*((60*60*24)^2/(1000^3));
#J2 = 1.08263e-3; %J2 spherical harmonic coefficient
#Re = 6378.1363/1000; %1000*km

rsm = ephemInterp(t,ephem[1],ephem[2],ephem[3],ephem[4],ephem[5],ephem[6]);
r_sun = rsm[1:3];
r_moon = rsm[4:6];

r = x[1:3];
rmag = sqrt.(r'*r);
rm = r-r_moon;
rmmag = sqrt.(rm'*rm);
rs = r-r_sun;
rsmag = sqrt.(rs'*rs);
rfmag = sqrt.(r_sun'*r_sun);

v = x[4:6];

a_earth = -mu_e*r/(rmag^3);
a_moon = -mu_m*rm/(rmmag^3);
a_sun = -mu_s*rs/(rsmag^3);
a_frame = -mu_s*r_sun/(rfmag^3);

a = a_earth + a_moon + a_sun + a_frame;

xdot[1:3] = v;
xdot[4:6] = a;

#da_earth = -(mu_e/(rmag^3))*eye(3) + (3*mu_e/(rmag^5))*(r*r');
#da_moon = -(mu_m/(rmmag^3))*eye(3) + (3*mu_m/(rmmag^5))*(rm*rm');
#da_sun = -(mu_s/(rsmag^3))*eye(3) + (3*mu_s/(rsmag^5))*(rs*rs');
#da = da_earth + da_moon + da_sun;
    
#A = [zeros(3) eye(3); da  zeros(3)];

#return (xdot, A)
    
end

dynamics (generic function with 1 method)

In [3]:
#This function reads in a .csv ephemeris file from the JPL horizons website
using CSV

function readEphemData(filename)
Data = CSV.read(filename);
t = Data[1][:]-Data[1][1]; #time in days
r1 = convert(Array{Float64},Data[3]); #km
r2 = convert(Array{Float64},Data[4]);
r3 = convert(Array{Float64},Data[5]);
v1 = convert(Array{Float64},Data[6]); #km/s
v2 = convert(Array{Float64},Data[7]);
v3 = convert(Array{Float64},Data[8]);

return (t, [r1 r2 r3 v1 v2 v3]');
end

readEphemData (generic function with 1 method)

In [4]:
#This function converts classical orbital elements to ECI state vectors
#Adapted from Astrodynamics by Vallado
function coe2rv(p,ecc,incl,omega,argp,nu);
mu_e = 398600.440;
small = 0.00000001;

#if ( ecc < small )
#    if (incl<small) | ( abs(incl-pi)< small )
#        argp = 0.0;
#        omega= 0.0;
#        nu   = truelon;
#    else
#        argp= 0.0;
#        nu  = arglat;
#    end
#else
#    if ( ( incl<small) | (abs(incl-pi)<small) )
#        argp = lonper;
#        omega= 0.0;
#    end
#end

cosnu= cos(nu);
sinnu= sin(nu);
temp = p / (1.0  + ecc*cosnu);
rpqw = [temp*cosnu, temp*sinnu, 0.0];
if ( abs(p) < 0.0001)
    p= 0.0001;
end
vpqw = [-sinnu*sqrt(mu_e)/sqrt(p), (ecc + cosnu)*sqrt(mu_e)/sqrt(p), 0.0];

tempvec = rot3(rpqw, -argp);
tempvec = rot1(tempvec, -incl);
r = rot3(tempvec, -omega);

tempvec = rot3(vpqw, -argp);
tempvec = rot1(tempvec, -incl);
v = rot3(tempvec, -omega);

return (r, v);
end

function rot1(vec, xval)
temp= vec[3];
c= cos(xval);
s= sin(xval);

return [vec[1], c*vec[2] + s*temp, c*vec[3] - s*vec[2]];
end

function rot3(vec, xval)
temp= vec[2];
c= cos(xval);
s= sin(xval);
    
return [c*vec[1] + s*temp, c*vec[2] - s*vec[1], vec[3]];
end

rot3 (generic function with 1 method)

In [7]:
#This is the script that sets up the simulation

#Load ephemeris for sun and moon from .csv files
(t_sun,x_sun) = readEphemData("sun_ephem.csv");
(t_moon,x_moon) = readEphemData("moon_ephem.csv");

t_ephem = t_sun;
x_ephem = [x_sun[1:3,:]/1000; x_moon[1:3,:]/1000]; #put distance in 1000*km

#Set up linear interpolation of sun + moon positions positions
xi1 = interpolate((t_ephem,), x_ephem[1,:], Gridded(Linear()));
xi2 = interpolate((t_ephem,), x_ephem[2,:], Gridded(Linear()));
xi3 = interpolate((t_ephem,), x_ephem[3,:], Gridded(Linear()));
xi4 = interpolate((t_ephem,), x_ephem[4,:], Gridded(Linear()));
xi5 = interpolate((t_ephem,), x_ephem[5,:], Gridded(Linear()));
xi6 = interpolate((t_ephem,), x_ephem[6,:], Gridded(Linear()));
xinterp = (xi1, xi2, xi3, xi4, xi5, xi6);

#Hub Initial Conditions
# 25 Jan 2024 22:30:25.520
a = 243729.554542; #km
e = 0.598335;
i = 51.975*(pi/180);
RAAN = 61.110*(pi/180);
argp = 247.345*(pi/180);
nu = 0.0; #true anomaly
p = a*(1-e*e); #semilatus rectum

#Convert to ECI state vector
(r0,v0) = coe2rv(p,e,i,RAAN,argp,nu);

#Convert units to 1000*km and days
x0hub = [r0/1000; v0*(24*60*60/1000)];
t0 = 25 + 22/24 + 30/(24*60) + 25.52/(24*60*60);

dt = 24/24; #Take measurements once every 24 hours
t_samp = t0:dt:(t0+14); #Run for 14 days

Nx = 6; #State dimension
N = length(t_samp); #Number of time steps

x0node = diagm(ones(Nx)+[1e-4*randn(3); 1e-5*randn(3)])*x0hub; #Generate random node initial conditions

#Propagate hub and node orbits (ground truth)
using DifferentialEquations
hub_prob = ODEProblem(dynamics,x0hub,(t_samp[1], t_samp[end]),xinterp);
hub_soln = solve(hub_prob,Tsit5(),reltol=1e-9,abstol=1e-9,saveat=dt);
node_prob = ODEProblem(dynamics,x0node,(t_samp[1], t_samp[end]),xinterp);
node_soln = solve(node_prob,Tsit5(),reltol=1e-9,abstol=1e-9,saveat=dt);

#Plot orbits
using Plots
plot(hub_soln[1,:],hub_soln[2,:],hub_soln[3,:],xshowaxis=false,yshowaxis=false,zshowaxis=false);
plot!(node_soln[1,:],node_soln[2,:],node_soln[3,:],xshowaxis=false,yshowaxis=false,zshowaxis=false);
scatter!([0.0],[0.0],[0.0],mcolor=:green)

In [8]:
#Plot true Hub-Node range
r = zeros(length(t_samp));
for k = 1:length(t_samp)
    r[k] = 1000*norm(node_soln[k][1:3] - hub_soln[k][1:3]);
end
plot(t_samp,r, xlabel="Time (Days)", ylabel="Hub-Node Range (km)")

In [9]:
#Generate noisy measurement data from ground truth simulations
function observation(xnode,xhub)
    y = norm(xnode[1:3]-xhub[1:3]);
end

yhist = zeros(length(hub_soln),1);
xnode = zeros(6,length(node_soln));
xhub = zeros(6,length(hub_soln));
for k = 1:length(hub_soln)
    xhub[:,k] = hub_soln[k];
    xnode[:,k] = node_soln[k];
    yhist[k] = observation(xnode[:,k], xhub[:,k]);
    
    yhist[k] += 1e-5*randn(); #Add noise: Std. Dev. = 10m in ranging
    xhub[:,k] += [1e-3*randn(3,1); 1e-3*randn(3,1)]; #Add noise: Std. Dev. = 1km in position and about 1 cm/sec in velocity
end

In [10]:
#Estimate full state trajectory

function lsobj2(z,t_samp,dt,xinterp,xhub,yhist)
    
    Qinv = 1e-3*eye(6); #This weights dynamics accuracy vs. measurement accuracy.
    
    N = length(yhist);
    
    #Number of RK4 steps to take between observations
    #I find that these steps should be no more than 3~4 hours long
    Nstages = 8;
    dts = dt/Nstages;
    
    xtraj = reshape(z[:],6,N);
    
    J = (yhist[1] - observation(xtraj[:,1], xhub[:,1])).^2;
    
    xd1 = xtraj[:,1];
    xd2 = xtraj[:,1];
    xd3 = xtraj[:,1];
    xd4 = xtraj[:,1];
    for k = 1:(length(t_samp)-1)
        
        xstage = xtraj[:,k];
        tstage = t_samp[k]
        for j = 1:Nstages
            dynamics(xd1,xstage,xinterp,tstage);
            dynamics(xd2,xstage+0.5*dts*xd1,xinterp,tstage+0.5*dts);
            dynamics(xd3,xstage+0.5*dts*xd2,xinterp,tstage+0.5*dts);
            dynamics(xd4,xstage+dts*xd3,xinterp,tstage+dts);
        
            xstage += (dts/6)*(xd1 + 2*xd2 + 2*xd3 + xd4);
            tstage += dts;
        end
        xp = xstage;
        
        #Cost function weights dyamics predictions vs. observations
        #This is the same Maximum a-posteriori cost function that a Kalman filter optimizes
        #But here we're treating it in its full nonlinear glory without making any linearizations
        #Or assumptions that the errors are Gaussian
        J += (xp-xtraj[:,k+1])'*Qinv*(xp-xtraj[:,k+1]) + (yhist[k+1]-observation(xtraj[:,k+1], xhub[:,k+1])).^2;
        
    end
    
    return J;
end

#Set up and solve nonlinear optimization problem
using Optim
f(x) = lsobj2(x, t_samp, dt, xinterp, hub_soln, yhist);
od = OnceDifferentiable(f, xnode[:]; autodiff = :forward);
fitresult = optimize(od, xnode[:]+0.1*randn(Nx*N), LBFGS(), Optim.Options(iterations = 2000))

Results of Optimization Algorithm
 * Algorithm: L-BFGS
 * Starting Point: [30.487193651882155,-59.905717540189706, ...]
 * Minimizer: [30.50296812736188,-59.90353481111018, ...]
 * Minimum: 2.936837e-10
 * Iterations: 2000
 * Convergence: false
   * |x - x'| ≤ 1.0e-32: false 
     |x - x'| = 8.41e-06 
   * |f(x) - f(x')| ≤ 1.0e-32 |f(x)|: false
     |f(x) - f(x')| = 2.16e-04 |f(x)|
   * |g(x)| ≤ 1.0e-08: false 
     |g(x)| = 1.71e-07 
   * Stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: true
 * Objective Calls: 5841
 * Gradient Calls: 5841

In [11]:
#Calculate and plot position errors in kilometers
e = zeros(length(t_samp));
xfit = reshape(Optim.minimizer(fitresult),6,length(t_samp));
for k = 1:length(t_samp)
    e[k] = 1000*norm(xfit[1:3,k] - xnode[1:3,k]);
end

plot(t_samp,e, xlabel="Time (Days)", ylabel="Position Error (km)")